# STEP 0: Install dependencies

In [27]:
!pip install -qU google-generativeai==0.8.5 google.ai.generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [28]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# b Secure Gemini API Key input

In [29]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Gemini API Key:")

Enter your Gemini API Key:··········


# STEP 2: Initialize Gemini 1.5 Flash

In [30]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2)

# STEP 3: Node to ask user for symptom

In [31]:
def get_symptom(state: dict) -> dict:
  symptom =input("Welcome to the XYZ hospital, Please enter a symptom: ")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [32]:
def classify_symptom(state: dict) -> dict:
  prompt = (
      "You are helpful medical assistant, Classify the symptoms below into of the categories:\n"
      "-General\n -Emergency\n -mental health\n \n"
      f"Symptom: {state['symptom']}\n"
      "Reply with only one word : General, Emergency, Mental Health"
      "#Example : input : I have fever : output : General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM Classifies the sympyom as: {category}") #debug
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [33]:
def symptom_router(state: dict) -> str:
  cat = state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency" in cat:
    return "emergency"
  elif "mental health" in cat:
    return "mental health"
  else:
    return "general"

# STEP 6: Category-specific response nodes


In [34]:
def general_node(state: dict) ->dict:
  state["answer"] = f"'{state['symptom']}' seems general: Directing you to general ward for consulting a doctor"
  return state

def emergency_node(state: dict) ->dict:
  state["answer"] = f"'{state['symptom']}' seems emergency: Directing you to emergency ward for consulting a doctor"
  return state

def mental_health_node(state: dict) ->dict:
  state["answer"] = f"'{state['symptom']}' seems mental health: Directing you to mental health ward for consulting a doctor"
  return state

# STEP 7: Build LangGraph


In [35]:
builder = StateGraph(dict)
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom) # Add get_symptom as a node
builder.add_node("classify_symptom", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental health", mental_health_node)


builder.add_edge("get_symptom", "classify_symptom")
builder.add_conditional_edges("classify_symptom", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "mental health": "mental health",
})
builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental health", END)

graph = builder.compile()

# STEP 8: Compile and invoke the graph


In [39]:
graph = builder.compile()
final_state = graph.invoke({})
print("Final Output: \n")
print(final_state["answer"])

Welcome to the XYZ hospital, Please enter a symptom: i having high pain
LLM Classifies the sympyom as: Emergency
Final Output: 

'i having high pain' seems emergency: Directing you to emergency ward for consulting a doctor
